In [92]:
import sys
sys.path.append('/Applications/anaconda/lib/python2.7/site-packages')

In [45]:
import numpy as np
import pandas as pd
from sklearn import cross_validation
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import grid_search
import xgboost as xgb

In [2]:
def y_transform(y):
    Y_sub = np.array([])
    for i in y:
        if i < 1.125:
            Y_sub = np.append(Y_sub,1)
        elif 1.125<=i<1.29:
            Y_sub = np.append(Y_sub,1.25)
        elif 1.29<=i<1.415:
            Y_sub = np.append(Y_sub,1.33)
        elif 1.415<=i<1.585:
            Y_sub = np.append(Y_sub,1.50)
        elif 1.585<=i<1.71:
            Y_sub = np.append(Y_sub,1.67)
        elif 1.71<=i<1.875:
            Y_sub = np.append(Y_sub,1.75)
        elif 1.875<=i<2.125:
            Y_sub = np.append(Y_sub,2.00)
        elif 2.125<=i<2.29:
            Y_sub = np.append(Y_sub,2.25)
        elif 2.29<=i<2.415:
            Y_sub = np.append(Y_sub,2.33)
        elif 2.415<=i<2.585:
            Y_sub = np.append(Y_sub,2.50)
        elif 2.585<=i<2.71:
            Y_sub = np.append(Y_sub,2.67)
        elif 2.71<=i<2.875:
            Y_sub = np.append(Y_sub,2.75)
        elif i>=2.875:
            Y_sub = np.append(Y_sub,3.00)
    return Y_sub

def fmean_squared_error(ground_truth, predictions):
    rmse = mean_squared_error(ground_truth, predictions)**0.5
    return rmse

In [37]:
feature = pd.read_csv('train_allfeature.csv')
feature.columns

Index([u'id', u'relevance', u'ts1', u'ts2', u'ts3', u'td1', u'td2', u'sd1',
       u'sd2', u'sd3', u'sd4', u'sd5', u'sd6', u'sd7', u'sd8', u'ts1_tfidf',
       u'ts2_tfidf', u'ts3_tfidf', u'td1_tfidf', u'td2_tfidf', u'sd1_tfidf',
       u'sd2_tfidf', u'sd3_tfidf', u'sd4_tfidf', u'sd5_tfidf', u'sd6_tfidf',
       u'sd7_tfidf', u'sd8_tfidf', u'brandDis', u'colorDis', u'materialDis'],
      dtype='object')

In [83]:
def mean(a):
    return np.array(a).mean()

## Random Forest

### Full Model 

In [96]:
RandomForestRegressor?

In [101]:
[i for i in range(20,30)]

[20, 21, 22, 23, 24, 25, 26, 27, 28, 29]

In [107]:
feature.shape

(74067, 31)

In [ ]:
RFRmseList = []
parameters = []
from sklearn.cross_validation import KFold
kf_total = cross_validation.KFold(len(feature.id), n_folds=5, shuffle=True, random_state=4)
for train, test in kf_total:
    x_train = feature.iloc[train,2:31]
    y_train = feature.iloc[train,1]
    x_test = feature.iloc[test,2:31]
    y_test = feature.iloc[test,1]
    
    RMSE=make_scorer(fmean_squared_error, greater_is_better=False)
    parameters = {'n_estimators':[i for i in range(25,35)],'max_depth':[i for i in range(5,15)]}
    clf = grid_search.GridSearchCV(RandomForestRegressor(),parameters,scoring=RMSE)
    clf.fit(x_train,y_train)
    
    y_pred = clf.predict(x_test)
    RFRmseList.append(fmean_squared_error(y_test,y_pred))
#     parameters.append(clf.best_params_)
    print clf.best_params_

In [112]:
mean(RFRmseList)

0.49343786079765967

### Distance

In [66]:
RFDisRmseList = []
from sklearn.cross_validation import KFold
kf_total = cross_validation.KFold(len(feature.id), n_folds=5, shuffle=True, random_state=4)
for train, test in kf_total:
    x_train = feature.iloc[train,2:14]
    y_train = feature.iloc[train,1]
    x_test = feature.iloc[test,2:14]
    y_test = feature.iloc[test,1]
    
    RMSE=make_scorer(fmean_squared_error, greater_is_better=False)
    parameters = {'n_estimators':[5,20],'max_depth':[2,10]}
    clf = grid_search.GridSearchCV(RandomForestRegressor(),parameters,scoring=RMSE)
    clf.fit(x_train,y_train)
    
    y_pred = clf.predict(x_test)
    RFDisRmseList.append(fmean_squared_error(y_test,y_pred))

In [85]:
mean(RFDisRmseList)

0.51038874160422931

### Tf-idf

In [68]:
RFTfRmseList = []
from sklearn.cross_validation import KFold
kf_total = cross_validation.KFold(len(feature.id), n_folds=5, shuffle=True, random_state=4)
for train, test in kf_total:
    x_train = feature.iloc[train,15:27]
    y_train = feature.iloc[train,1]
    x_test = feature.iloc[test,15:27]
    y_test = feature.iloc[test,1]
    
    RMSE=make_scorer(fmean_squared_error, greater_is_better=False)
    parameters = {'n_estimators':[5,20],'max_depth':[2,10]}
    clf = grid_search.GridSearchCV(RandomForestRegressor(),parameters,scoring=RMSE)
    clf.fit(x_train,y_train)
    
    y_pred = clf.predict(x_test)
    RFTfRmseList.append(fmean_squared_error(y_test,y_pred))

In [86]:
mean(RFTfRmseList)

0.49672086150292216

### Attributes

In [70]:
RFAttrRmseList = []
from sklearn.cross_validation import KFold
kf_total = cross_validation.KFold(len(feature.id), n_folds=5, shuffle=True, random_state=4)
for train, test in kf_total:
    x_train = feature.iloc[train,28:30]
    y_train = feature.iloc[train,1]
    x_test = feature.iloc[test,28:30]
    y_test = feature.iloc[test,1]
    
    RMSE=make_scorer(fmean_squared_error, greater_is_better=False)
    parameters = {'n_estimators':[5,20],'max_depth':[2,10]}
    clf = grid_search.GridSearchCV(RandomForestRegressor(),parameters,scoring=RMSE)
    clf.fit(x_train,y_train)
    
    y_pred = clf.predict(x_test)
    RFAttrRmseList.append(fmean_squared_error(y_test,y_pred))

In [87]:
mean(RFAttrRmseList)

0.53260150528737848

### Distance & Tf-idf

In [93]:
RFDTRmseList = []
from sklearn.cross_validation import KFold
kf_total = cross_validation.KFold(len(feature.id), n_folds=5, shuffle=True, random_state=4)
for train, test in kf_total:
    x_train = feature.iloc[train,2:27]
    y_train = feature.iloc[train,1]
    x_test = feature.iloc[test,2:27]
    y_test = feature.iloc[test,1]
    
    RMSE=make_scorer(fmean_squared_error, greater_is_better=False)
    parameters = {'n_estimators':[5,20],'max_depth':[2,10]}
    clf = grid_search.GridSearchCV(RandomForestRegressor(),parameters,scoring=RMSE)
    clf.fit(x_train,y_train)
    
    y_pred = clf.predict(x_test)
    RFDTRmseList.append(fmean_squared_error(y_test,y_pred))

In [94]:
RFDTRmseList

[0.49418513725556773,
 0.49812348666840911,
 0.49430587121822051,
 0.48964869490274482,
 0.49629828978479001]

In [95]:
mean(RFDTRmseList)

0.49451229596594642

## XGBoosting

### Full model

In [97]:
xgb.XGBRegressor?

In [72]:
XGBRmseList = []
from sklearn.cross_validation import KFold
kf_total = cross_validation.KFold(len(feature.id), n_folds=5, shuffle=True, random_state=4)
for train, test in kf_total:
    x_train = feature.iloc[train,2:30]
    y_train = feature.iloc[train,1]
    x_test = feature.iloc[test,2:30]
    y_test = feature.iloc[test,1]
    
    xgb_model = xgb.XGBRegressor(learning_rate=0.25, silent=False, objective="reg:linear", nthread=-1, gamma=0, min_child_weight=1, max_delta_step=0,
                     subsample=1, colsample_bytree=1, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                     base_score=0.5, seed=0, missing=None)

    param_grid = {'xgb_model__max_depth': [5,20], 'xgb_model__n_estimators': [2,10]}
    clf = grid_search.GridSearchCV(xgb_model,parameters,scoring=RMSE)
    clf.fit(x_train,y_train)
    
    y_pred = clf.predict(x_test)
    XGBRmseList.append(fmean_squared_error(y_test,y_pred))

In [88]:
mean(XGBRmseList)

0.49728636029847328

### Distance

In [74]:
XGBDisRmseList = []
from sklearn.cross_validation import KFold
kf_total = cross_validation.KFold(len(feature.id), n_folds=5, shuffle=True, random_state=4)
for train, test in kf_total:
    x_train = feature.iloc[train,2:14]
    y_train = feature.iloc[train,1]
    x_test = feature.iloc[test,2:14]
    y_test = feature.iloc[test,1]
    
    xgb_model = xgb.XGBRegressor(learning_rate=0.25, silent=False, objective="reg:linear", nthread=-1, gamma=0, min_child_weight=1, max_delta_step=0,
                     subsample=1, colsample_bytree=1, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                     base_score=0.5, seed=0, missing=None)

    param_grid = {'xgb_model__max_depth': [5,20], 'xgb_model__n_estimators': [2,10]}
    clf = grid_search.GridSearchCV(xgb_model,parameters,scoring=RMSE)
    clf.fit(x_train,y_train)
    
    y_pred = clf.predict(x_test)
    XGBDisRmseList.append(fmean_squared_error(y_test,y_pred))

In [89]:
mean(XGBDisRmseList)

0.5104646879729362

### Tf-idf

In [76]:
XGBTfRmseList = []
from sklearn.cross_validation import KFold
kf_total = cross_validation.KFold(len(feature.id), n_folds=5, shuffle=True, random_state=4)
for train, test in kf_total:
    x_train = feature.iloc[train,15:27]
    y_train = feature.iloc[train,1]
    x_test = feature.iloc[test,15:27]
    y_test = feature.iloc[test,1]
    
    xgb_model = xgb.XGBRegressor(learning_rate=0.25, silent=False, objective="reg:linear", nthread=-1, gamma=0, min_child_weight=1, max_delta_step=0,
                     subsample=1, colsample_bytree=1, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                     base_score=0.5, seed=0, missing=None)

    param_grid = {'xgb_model__max_depth': [5,20], 'xgb_model__n_estimators': [2,10]}
    clf = grid_search.GridSearchCV(xgb_model,parameters,scoring=RMSE)
    clf.fit(x_train,y_train)
    
    y_pred = clf.predict(x_test)
    XGBTfRmseList.append(fmean_squared_error(y_test,y_pred))

In [90]:
mean(XGBTfRmseList)

0.49834686231411018

### Attributes

In [78]:
XGBAttrRmseList = []
from sklearn.cross_validation import KFold
kf_total = cross_validation.KFold(len(feature.id), n_folds=5, shuffle=True, random_state=4)
for train, test in kf_total:
    x_train = feature.iloc[train,28:30]
    y_train = feature.iloc[train,1]
    x_test = feature.iloc[test,28:30]
    y_test = feature.iloc[test,1]
    
    xgb_model = xgb.XGBRegressor(learning_rate=0.25, silent=False, objective="reg:linear", nthread=-1, gamma=0, min_child_weight=1, max_delta_step=0,
                     subsample=1, colsample_bytree=1, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                     base_score=0.5, seed=0, missing=None)

    param_grid = {'xgb_model__max_depth': [5,20], 'xgb_model__n_estimators': [2,10]}
    clf = grid_search.GridSearchCV(xgb_model,parameters,scoring=RMSE)
    clf.fit(x_train,y_train)
    
    y_pred = clf.predict(x_test)
    XGBAttrRmseList.append(fmean_squared_error(y_test,y_pred))

In [91]:
mean(XGBAttrRmseList)

0.53233330795313405

## Decision Tree

In [14]:
from sklearn.tree import DecisionTreeRegressor

In [24]:
RMSE=make_scorer(fmean_squared_error, greater_is_better=False)
parameters = {'max_leaf_nodes':[2,20],'max_depth':[2,20]}
clf = grid_search.GridSearchCV(DecisionTreeRegressor(),parameters,scoring=RMSE)
clf.fit(x_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_leaf_nodes': [2, 20], 'max_depth': [2, 20]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring=make_scorer(fmean_squared_error, greater_is_better=False),
       verbose=0)

In [25]:
y_pred = clf.predict(x_test)
fmean_squared_error(y_test,y_pred)

0.5114248375170225